# Llama2 Peft 파인튜닝

## AutoTrain Advanced

코드 한 줄로 파인튜닝 가능하게 하는 패키지

https://github.com/huggingface/autotrain-advanced

In [ ]:
!pip install -q autotrain-advanced # AUTOTRAIN 툴을 사용하여 LLN을 학습시킵니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 102.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 

## PyTorch 업데이트

> Colab에서 사용하기 위해서는 필수

In [ ]:
!autotrain setup --update-torch # 파이토치를 최신버전으로 업그래이드합니다.

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


## Finetune

A100 7.5h/epoch

- 기본 모델: https://huggingface.co/TinyPixel/Llama-2-7B-bf16-sharded
- KoAlpaca 데이터셋(한국어): https://huggingface.co/datasets/royboy0416/ko-alpaca
- OpenAssistant 데이터셋(영어): https://huggingface.co/datasets/timdettmers/openassistant-guanaco
- 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21

In [ ]:
!autotrain llm --train \
    --project_name "llama2-finetune" \
    --model "TinyPixel/Llama-2-7B-bf16-sharded" \
    --data_path "minecode/adfa" \
    --text_column "text" \
    --use_peft \
    --use_int4 \
    --learning_rate 2e-4 \
    --train_batch_size 5 \
    --num_train_epochs 40 \
    --trainer sft \
    --model_max_length 2048

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.0.1+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='minecode/adfa', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', model='TinyPixel/Llama-2-7B-bf16-sharded', learning_rate=0.0002, num_train_epochs=40, train_batch_size=5, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_

## Google Drive 저장

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!cp -a /content/llama2-finetune /content/drive/MyDrive

## Inference

In [ ]:
!pip install -q transformers peft bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
peft_model_id = "/content/drive/MyDrive/llama2-finetune/checkpoint-1560"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False, # 8bit 로드 X
    load_in_4bit=True, # 4bit 로드를 사용.
    llm_int8_threshold=6.0, # 6.0 이상의 값은 8bit로 로드
    llm_int8_skip_modules=None, 
    llm_int8_enable_fp32_cpu_offload=False, # CPU 오프로드 사용 X
    llm_int8_has_fp16_weight=False, # FP16 가중치 사용 X
    bnb_4bit_quant_type="nf4", # NF4 사용
    bnb_4bit_use_double_quant=False, # 이중 양자화 사용 X
    bnb_4bit_compute_dtype="float16", # 4bit 연산은 FP16 사용
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 4bit 모델 로드
model = PeftModel.from_pretrained(model, peft_model_id) # Peft 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_id) # 토크나이저 로드

model.eval() # 모델 평가.

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, 

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

def gen(x):
    q = prompt % (x,) # 프롬프트 생성
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt', # 텐서로 반환
            return_token_type_ids=False # 토큰 타입 아이디 반환 X
        ).to('cuda'), # GPU 사용
        max_new_tokens=1000, # 최대 1000개 토큰 생성
        early_stopping=True, # 조건에 맞으면 중단
        do_sample=True, # 샘플링 사용
    )
    return tokenizer.decode(gened[0]).replace(q, "")

In [ ]:
gen("국어 - 듣기·말하기 ([10국01-01]): 개인이나 집단에 따라 듣기와 말하기의 방법이 다양함을 이해하고 듣기·말하기 활동을 한다. 이것을 보고 수행평가를 작성해줘.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'<s> 국어 - 듣기·디코러 [10국01-01] 를 위해 개인이나 집단에 따라 듣기와 말하기의 방법이 다양함을 이해하고 듣기·디코러 활동을 한 내용을 작성하십시오. 이를 바탕으로 수행평가를 작성하되, 주제에 대한 전반적인 도움이 필요할 때는 다음의 준법적 질문을 통해 정보를 얻고 출발점으로 차용하십시오. : Query:   듣기·디코러 활동에서 어떤 내용을 다양하게 접근하였습니까?   듣기·디코러 활동을 통해 어떤 경험을 했습니까?   어떤 부분에 어떤 문제가 있었고 서로를 배울 수 있었는지 서로에게 보여주시고 설정설정하십시오. : Response: 우선, 듣기·디코러 활동에서는 다양한 이론과 방법을 접근하였습니다. 예를 들어, 팔로워에 대한 영향 강하지 않는 이유로 팔로워 수를 증가시키는 방법을 학습하였습니다. 또한, 서로의 의견을 존중하고 이해하는 방법을 배운습니다. 이 과정에서 제가  наи았던 문제는 정보의 적정성에 대한 것이었습니다. 항상 정확한 정보를 지니지 않았고, 모방했을 때 오류가 나기 쉬웠습니다. 이로 인해 서로를 더 잘 이해할 수 없었습니다. 이런 경험을 통해 듣기·디코러의 중요성을 더욱 깨닫게 되었습니다. 듣기와 디코러는 서로에게 영향을 주는 행위이며, 이를 잘 하지 않으면 결과적으로 자신의 목표를 달성할 수 없습니다. 이에 대해 이해를 높이는 데 큰 도움이 되었습니다. 마'

In [20]:
gen("사과는 무슨 맛인가요?")

'<s> 사과는 겨울을 혈류로 고쳐 ジ gives 제대 우리 사랑하는 귀화 하는 거 한국 농과는 다른 행성 있습니다. 환경과 이별하지 않고   필요한 열기어와 물을 제공하며   생활을 이어가고 있습니다. 이러한 행성의 생활 방식을 노담에 반영하여   사과를 떠올려 보내주세요.  ### Input: 사과는 겨울을 혈류로 고쳐 주는 행성 입니다. 그럼 겨울을 혈류로 고칩니다. 그 후   새로운 맛을 찾아보며   사과의 진정한 가치를 배우며 죽임.  ### Response: 사과는 겨울을 혈류로 고치는 행성 사윌 필요한 열기어와 물을 제공하며   생활을 이어간다. 이러한 생활 방식을 노담에 반영하여   사과를 떠올려 보며   바로 잡아먹음. 환경을 과시하고   사랑하며   생활을 이어가는 행성사윌 필요한 열기어와 물을 제공하며   생활을 이어가고 있습니다. 이러한 행 stars의 생활 방식을 노담에 반영하여   사과를 떠올려 보며   바로 잡아먹음. 환경을 과시하고   사랑하며   생활을 이어가는 행성사윌.</s>'

In [21]:
gen("인생에서 무엇이 가장 중헌가요?")

'<s> 인생에서 가장 중요한 것은 서로에게 존중과 이해의 변형이라는 것입니다. 그러므로 인생의 중황은 그러한 감사와 이해를 통해 왔다고 할 수 있습니다. 인생에서 가장 중헌한 것은 서로에게 존중과 이해의 변형이었습니다. 예를 들어   친구의 문제에 도움을 준 것   아내에게 존중을 보냄게 존중을 보냄게 이해한 것   그리고 자신의 잘못을 자신의 잘몰을 인식하게 된 것입니다. 이런 경험를 통해 생각과 행동의 음악인 인생에서는 가장 중헌한 것은 서로에게 존중과 이해의 변형입니다. 이는 우리가 他인과 갈등하는 상황에서도 서로에게 존중하고 이해하는 데 중요한 것을 가르쳐 줍니다. 이 is 인생의 가장 중헌한 것이입니다.</s>'